In [106]:
import numpy as np
import pandas as pd
import altair as alt
from statsmodels.tsa.api import Holt
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_row', 100)
path = '/Users/ephraimgoldin/Dropbox/OLG Model 2/'

jh_hubei_data = pd.read_csv(path + 'jh_hubei_data.csv').values 
#countrydata_df = pd.read_csv(path + 'countries_data.csv', parse_dates=['date'])
countrydata_df = pd.read_csv('https://raw.githubusercontent.com/gstat-gcloud/covid19-sim/master/Resources/Datasets/CountryData/all_dates.csv',
                             parse_dates=['date'])
stringency = pd.read_csv(path + 'stringencyExample.csv', usecols=['date', 'StringencyIndex'], parse_dates=['date'])
#from scipy.io import loadmat
#mat = loadmat(path +'sample.mat')

In [3]:
def process(self, init_infected, detected=None):
        if detected is None:
            detected = self.df_tmp['total_cases'].values

        day_0 = np.argmax(detected >= init_infected)
        self.day_0 = day_0
        detected = detected[day_0:]
        self.detected = [detected[0]]
        for t in range(1, len(detected)):
            self.detected.append(max(detected[t - 1] + 1, detected[t]))

        if not self.df_tmp.empty:
            self.df_tmp = self.df_tmp[day_0:]
            self.df_tmp['StringencyIndex'].fillna(method='ffill', inplace=True)
class Parameters:
    def __init__(self, tau, init_infected, fi, theta, countries, critical_condition_rate, recovery_rate,
            critical_condition_time, recovery_time):
        self.tau = tau
        self.init_infected = init_infected
        self.fi = fi  # proportion of infectives are never diagnosed
        self.theta = theta  # diagnosis daily rate

        self.countries = countries
        self.critical_condition_rate = critical_condition_rate
        self.recovery_rate = recovery_rate
        self.critical_condition_time = critical_condition_time
        self.recovery_time = recovery_time

            

In [6]:
p = Parameters(tau=8,
               init_infected=50,
               fi=0.25,
               theta=0.0771,
               countries=['israel'],
               critical_condition_rate=0.051,
               recovery_rate=0.4,
               critical_condition_time=10,
               recovery_time=6
               )


In [122]:
init_infected = p.init_infected
df = countrydata_df[countrydata_df['total_cases'] > init_infected]
df = df.reset_index(drop=True)
#day0 =  df.groupby(['country'])['date'].min() 
df["day0"] =  df.groupby(['country'])['date'].transform('min')
df['coronadays'] = (df['date'] - df['day0']).dt.days
df["maxcdays"] =  df.groupby(['country'])['coronadays'].transform('max')
ILday = df[df['country']=='israel'][['coronadays']].max()
print(ILday.values)
dfcluster = df[df['coronadays'].values  == ILday.values]
print(dfcluster.columns) 
dfc = dfcluster[[ 
       'serious_critical', 'tot_cases/1m_pop', 'deaths/1m_pop',
       'tests/_1m_pop']]

X_fitted  = dfc.values
kmeans = KMeans(n_clusters=4, random_state=0).fit_predict(X_fitted)
dfcluster['cluster'] = kmeans
dfcluster[['country','cluster','maxcdays']]


[48.]
Index(['Unnamed: 0', 'Unnamed: 0.1', 'country', 'total_cases', 'new_cases',
       'total_deaths', 'new_deaths', 'total_recovered', 'activecases',
       'serious_critical', 'tot_cases/1m_pop', 'deaths/1m_pop',
       'reported1st_case', 'ref', 'date', 'totaltests', 'tests/_1m_pop',
       'region', 'tot_deaths/1m_pop', 'population', 'S', 'E', 'I', 'R',
       'CountryCode', 'S1_School closing', 'S1_IsGeneral', 'S1_Notes',
       'S2_Workplace closing', 'S2_IsGeneral', 'S2_Notes',
       'S3_Cancel public events', 'S3_IsGeneral', 'S3_Notes',
       'S4_Close public transport', 'S4_IsGeneral', 'S4_Notes',
       'S5_Public information campaigns', 'S5_IsGeneral', 'S5_Notes',
       'S6_Restrictions on internal movement', 'S6_IsGeneral', 'S6_Notes',
       'S7_International travel controls', 'S7_Notes', 'S8_Fiscal measures',
       'S8_Notes', 'S9_Monetary measures', 'S9_Notes',
       'S10_Emergency investment in health care', 'S10_Notes',
       'S11_Investment in Vaccines', 'S11_

/Users/ephraimgoldin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,country,cluster,maxcdays
1844,china,0,77.0
1850,japan,0,77.0
2213,singapore,0,74.0
2274,hong kong,0,74.0
2822,diamond princess,0,70.0
3050,south korea,0,68.0
3464,italy,2,65.0
3752,united states,0,63.0
3775,iran,0,63.0
4363,france,0,59.0


In [115]:
dfcluster

,Unnamed: 0,Unnamed: 0.1,country,total_cases,new_cases,total_deaths,new_deaths,total_recovered,activecases,serious_critical,tot_cases/1m_pop,deaths/1m_pop,reported1st_case,ref,date,...,S10_Notes,S11_Investment in Vaccines,S11_Notes,S12_Testing framework,S12_Notes,S13_Contact tracing,S13_Notes,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,Unnamed: 39,day0,coronadays,cluster
1844,4721,4721.0,china,81439.0,0,3300.0,0,75448.0,2691.0,742.0,57.0,2.0,Jan 10,https://web.archive.org/web/20200329/https://w...,2020-03-29,...,NaN,0.0,NaN,2.0,China adopted a policy of early identification...,2.0,NaN,82059.0,3304.0,66.67,66.67,\t,2020-02-10,48.0,0
1850,4774,4774.0,japan,1866.0,173,54.0,2,424.0,1388.0,56.0,15.0,0.4,Jan 14,https://web.archive.org/web/20200329/https://w...,2020-03-29,...,NaN,0.0,NaN,1.0,NaN,1.0,NaN,1693.0,52.0,61.90,61.90,\t,2020-02-10,48.0,0
2213,5367,5367.0,singapore,1000.0,74,3.0,0,245.0,752.0,24.0,171.0,0.5,Jan 22,https://web.archive.org/web/20200401/https://w...,2020-04-01,...,NaN,0.0,NaN,2.0,"Singapore tests anyone eligible, not have to b...",2.0,we only quarantine the close contacts of confi...,879.0,3.0,42.86,42.86,\t,2020-02-13,48.0,0
2274,5428,5428.0,hong kong,765.0,50,4.0,0,147.0,614.0,5.0,102.0,0.5,Jan 22,https://web.archive.org/web/20200401/https://w...,2020-04-01,...,NaN,0.0,NaN,2.0,NaN,2.0,NaN,765.0,4.0,85.71,85.71,\t,2020-02-13,48.0,0
2822,6271,6271.0,diamond princess,712.0,0,11.0,0,619.0,82.0,10.0,0.0,0.0,0,https://web.archive.org/web/20200405/https://w...,2020-04-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-02-17,48.0,0
3050,6643,6643.0,south korea,10331.0,47,192.0,6,6694.0,3445.0,55.0,202.0,4.0,0,https://web.archive.org/web/20200407/https://w...,2020-04-07,...,NaN,0.0,NaN,3.0,KCDC- The newly developed diagnostic will be a...,2.0,"Extensive public information campaign, most no...",10331.0,192.0,80.95,80.95,\t,2020-02-19,48.0,0
3464,7266,7266.0,italy,147577.0,3951,18849.0,570,30455.0,98273.0,3497.0,2441.0,312.0,0,https://web.archive.org/web/20200410/https://w...,2020-04-10,...,NaN,0.0,NaN,2.0,Source: https://web.archive.org/web/2020041220...,2.0,https://web.archive.org/web/20200412201728/htt...,143626.0,18281.0,95.24,95.24,\t,2020-02-22,48.0,2
3752,7691,7691.0,united states,560246.0,27367,22101.0,1524,32237.0,505908.0,11766.0,1693.0,67.0,0,https://web.archive.org/web/20200412/https://w...,2020-04-12,...,NaN,0.0,NaN,3.0,NaN,1.0,'Contact tracing was used in Seattle and parts...,529951.0,20608.0,76.19,76.19,\t,2020-02-24,48.0,0
3775,7714,7714.0,iran,71686.0,1657,4474.0,117,43894.0,23318.0,3930.0,853.0,53.0,0,https://web.archive.org/web/20200412/https://w...,2020-04-12,...,NaN,0.0,NaN,0.0,All the news reports available in English seem...,0.0,No word on any contact tracing measures till 2...,70029.0,4357.0,66.67,66.67,\t,2020-02-24,48.0,0
4363,8544,8544.0,france,165027.0,17164,17920.0,753,32812.0,114295.0,6248.0,2528.0,275.0,0,https://web.archive.org/web/20200416/https://w...,2020-04-16,...,NaN,0.0,NaN,1.0,Complete lockdown\nhttp://web.archive.org/web/...,0.0,Complete lockdown\nhttp://web.archive.org/web/...,106206.0,17167.0,100.00,100.00,\t,2020-02-28,48.0,0


In [116]:
dfcluster[['country','cluster']]

,country,cluster
1844,china,0
1850,japan,0
2213,singapore,0
2274,hong kong,0
2822,diamond princess,0
3050,south korea,0
3464,italy,2
3752,united states,0
3775,iran,0
4363,france,0
